In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import xgboost as xgb
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

train2=pd.read_csv("train2.csv")
test=pd.read_csv("test2.csv")

zero_features = ['education', 'engnat', 'hand', 'married']

index = test['index']

pd.set_option('display.max_columns', 100)

Unneed_list = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE','QfE', 'QgE', 'QhE', 'QiE', 'QjE',
               'QkE', 'QlE', 'QmE', 'QnE', 'QoE','QpE', 'QqE', 'QrE', 'QsE', 'QtE', 'index']
train2.drop(Unneed_list, axis=1, inplace=True)
test.drop(Unneed_list, axis=1, inplace=True)


train3=pd.get_dummies(train2)
test=pd.get_dummies(test)


zero_features = ['education', 'engnat', 'hand', 'married']

#결손값(무응답 = 0)제거 함수
def drop_NoResponse(df):
    for col in zero_features:
        df.drop(df[df[col]==0].index, axis=0, inplace=True)
        
    return df
train = drop_NoResponse(train3)



max_index = train['familysize'].idxmax()
train.drop(max_index, axis=0, inplace=True)
#trainX에 투표값 제거
train_X = train.drop('voted', axis = 1)

#trainy에 투표값만 넣음
train_y = train['voted']


X_train, X_test, y_train, y_test=train_test_split(train_X, train_y, test_size=0.2, random_state=0)

rf_clf = LGBMClassifier(random_state = 0)
params = { 'n_estimators' : [100, 200],
         'max_depth' : [2, 4, 6, 8, 10]
         }

grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, refit=True, scoring = 'f1')
grid_cv.fit(X_train, y_train)
pred = grid_cv.predict(X_test)

#적용 결과 확인
scores_rf=pd.DataFrame(grid_cv.cv_results_)
scores_rf[['params', 'mean_test_score', 'rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 2, 'n_estimators': 100}",0.685015,5,0.680121,0.681232,0.680767
1,"{'max_depth': 2, 'n_estimators': 200}",0.681662,10,0.678167,0.680643,0.677752
2,"{'max_depth': 4, 'n_estimators': 100}",0.692468,1,0.691424,0.691031,0.686662
3,"{'max_depth': 4, 'n_estimators': 200}",0.683981,6,0.684400,0.683214,0.679299
4,"{'max_depth': 6, 'n_estimators': 100}",0.690150,3,0.688271,0.685836,0.686254
5,"{'max_depth': 6, 'n_estimators': 200}",0.683320,7,0.684305,0.679103,0.678661
6,"{'max_depth': 8, 'n_estimators': 100}",0.688277,4,0.686551,0.687169,0.687190
7,"{'max_depth': 8, 'n_estimators': 200}",0.682804,9,0.679743,0.681149,0.683250
8,"{'max_depth': 10, 'n_estimators': 100}",0.690850,2,0.685634,0.687379,0.683978
9,"{'max_depth': 10, 'n_estimators': 200}",0.683001,8,0.680236,0.680331,0.677984
